In [1]:
import pandas as pd
import numpy as np

In [2]:
# read students data csv file
file="raw_data/students_complete.csv"
file1="raw_data/schools_complete.csv"
df_students=pd.read_csv(file)
df_schools=pd.read_csv(file1)
df_students.head()

,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [3]:
df_schools.head()

,School ID,name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


**District Summary**

* Create a high level snapshot (in table form) of the district's key metrics, including:
  * Total Schools
  * Total Students
  * Total Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)


In [4]:
district_schools_df=df_schools.groupby('type')
no_schools=district_schools_df['name'].agg(np.size)
size_budget=district_schools_df['size','budget'].agg(np.sum)
district_summary=size_budget
district_summary['Number of Schools']=no_schools
district_summary.reset_index(level=0, inplace=True)

In [5]:
merged_df=pd.merge(df_schools,df_students,left_on="name",right_on="school",how="outer",suffixes=('_school','_student'))
merged_df['pass_math']=merged_df['math_score']>=70
merged_df['pass_reading']=merged_df['reading_score']>=70

merged_df.head()

,School ID,name_school,type,size,budget,Student ID,name_student,gender,grade,school,reading_score,math_score,pass_math,pass_reading
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,Huang High School,66,79,True,False
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,Huang High School,94,61,False,True
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,Huang High School,90,60,False,True
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,Huang High School,67,58,False,False
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,Huang High School,97,84,True,True


In [6]:
district_students=merged_df.groupby('type')
no_students=district_students['name_student'].agg(np.size)
score_data=district_students['reading_score','math_score'].agg(np.mean)
score_data.reset_index(level=0, inplace=True)
pass_nos=district_students['pass_math','pass_reading'].agg(np.sum)
pass_nos.reset_index(level=0, inplace=True)

district_summary1=pd.merge(district_summary,score_data,on="type")
district_summary2=pd.merge(district_summary1,pass_nos,on="type")
# district_summary2.append(no_students)
# pass_read=district_students['pass_reading'].value_counts()
# district_summary['reading_score','math_score']=score_data
# district_summary['pass_math','pass_reading']=pass_nos

In [7]:
no_students_d=no_students.to_frame()
no_students_d.reset_index(level=0, inplace=True)
no_students_d

,type,name_student
0,Charter,12194
1,District,26976


In [8]:
district_summary3=pd.merge(district_summary2,no_students_d,on="type")

In [9]:
sum_students=district_summary3['name_student'].sum()
district_summary3['pass_math']=district_summary3['pass_math']/sum_students*100
district_summary3['pass_reading']=district_summary3['pass_reading']/sum_students*100
district_summary3['Overall Passing Rate']=(district_summary3['pass_math']+district_summary3['pass_reading'])/2
district_summary3.columns=["type","Total Students","Total Budget","Total Schools","Average Math Score","Average Reading Score",
                          "% Passing Math","% Passing Reading","no_students","Overall Passing Rate"]
del district_summary3['no_students']
district_summary4=district_summary3[["type","Total Schools","Total Budget","Average Math Score","Average Reading Score",
                                   "% Passing Math","% Passing Reading","Overall Passing Rate"]]

# District Summary

In [10]:
district_summary4.set_index('type')

,Total Schools,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
type,,,,,,,
Charter,8,7301505,83.902821,83.406183,29.170283,30.086801,29.628542
District,7,17347923,80.962485,76.987026,45.810569,55.718662,50.764616


**School Summary**

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)


In [31]:
df_students['pass_math']=df_students['math_score']>=70
df_students['pass_reading']=df_students['reading_score']>=70

studentsdf_grouped=df_students.groupby('school')
score_summary=studentsdf_grouped["reading_score","math_score"].agg([np.mean])
score_summary.columns=['Average Reading Score','Average Math Score']
score_summary.reset_index(level=0,inplace=True)
t_students=studentsdf_grouped['reading_score'].agg(np.size)
# t_students.reset_index(level=0,inplace=True)
t_students1=t_students.to_frame(name="total students")


pass_summary=studentsdf_grouped["pass_math","pass_reading"].agg(np.sum)
pass_summary['% Passing Math']=pass_summary['pass_math']/(t_students1['total students'])*100


pass_summary['% Passing Reading']=pass_summary['pass_reading']/(t_students1['total students'])*100
del pass_summary['pass_math']
del pass_summary['pass_reading']
pass_summary['Overall Passing Rate']=(pass_summary["% Passing Math"]+pass_summary["% Passing Reading"])/2
pass_summary.reset_index(level=0,inplace=True)


In [34]:
test_score=pd.merge(score_summary,pass_summary,on="school")
# test_score

In [35]:
school_summary_df=pd.merge(df_schools,test_score,left_on="name",right_on="school")
del school_summary_df["School ID"];del school_summary_df['school']
school_summary_df1=school_summary_df.rename(columns={"name":"School Name",
                                 "type":"School Type",
                                 "size":"Total Students",
                                  "budget":"Total Budget"
                                 })
school_summary_df1['Per Student Budget']=school_summary_df1['Total Budget']/school_summary_df1['Total Students']

# School Summary

In [36]:
school_summary_df1.set_index('School Name',drop=True,inplace=True)
school_summary_df1

,School Type,Total Students,Total Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,Overall Passing Rate,Per Student Budget
School Name,,,,,,,,,
Huang High School,District,2917,1910635,81.182722,76.629414,65.683922,81.316421,73.500171,655.0
Figueroa High School,District,2949,1884411,81.158020,76.711767,65.988471,80.739234,73.363852,639.0
Shelton High School,Charter,1761,1056600,83.725724,83.359455,93.867121,95.854628,94.860875,600.0
Hernandez High School,District,4635,3022020,80.934412,77.289752,66.752967,80.862999,73.807983,652.0
Griffin High School,Charter,1468,917500,83.816757,83.351499,93.392371,97.138965,95.265668,625.0
Wilson High School,Charter,2283,1319574,83.989488,83.274201,93.867718,96.539641,95.203679,578.0
Cabrera High School,Charter,1858,1081356,83.975780,83.061895,94.133477,97.039828,95.586652,582.0
Bailey High School,District,4976,3124928,81.033963,77.048432,66.680064,81.933280,74.306672,628.0
Holden High School,Charter,427,248087,83.814988,83.803279,92.505855,96.252927,94.379391,581.0


# Top Performing Schools (By Overall Passing Rate)

In [38]:
school_summary_df1.sort_values("Overall Passing Rate",ascending=False).head()

,School Type,Total Students,Total Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,Overall Passing Rate,Per Student Budget
School Name,,,,,,,,,
Cabrera High School,Charter,1858,1081356,83.975780,83.061895,94.133477,97.039828,95.586652,582.0
Thomas High School,Charter,1635,1043130,83.848930,83.418349,93.272171,97.308869,95.290520,638.0
Pena High School,Charter,962,585858,84.044699,83.839917,94.594595,95.945946,95.270270,609.0
Griffin High School,Charter,1468,917500,83.816757,83.351499,93.392371,97.138965,95.265668,625.0
Wilson High School,Charter,2283,1319574,83.989488,83.274201,93.867718,96.539641,95.203679,578.0


# Bottom Performing Schools (By Overall Passing Rate)

In [39]:
school_summary_df1.sort_values("Overall Passing Rate").head()

,School Type,Total Students,Total Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,Overall Passing Rate,Per Student Budget
School Name,,,,,,,,,
Rodriguez High School,District,3999,2547363,80.744686,76.842711,66.366592,80.220055,73.293323,637.0
Figueroa High School,District,2949,1884411,81.158020,76.711767,65.988471,80.739234,73.363852,639.0
Huang High School,District,2917,1910635,81.182722,76.629414,65.683922,81.316421,73.500171,655.0
Johnson High School,District,4761,3094650,80.966394,77.072464,66.057551,81.222432,73.639992,650.0
Ford High School,District,2739,1763916,80.746258,77.102592,68.309602,79.299014,73.804308,644.0


* Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.


In [40]:
df_students.head()

,Student ID,name,gender,grade,school,reading_score,math_score,pass_math,pass_reading
0,0,Paul Bradley,M,9th,Huang High School,66,79,True,False
1,1,Victor Smith,M,12th,Huang High School,94,61,False,True
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,False,True
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,False,False
4,4,Bonnie Ray,F,9th,Huang High School,97,84,True,True


# Average Math Score at each Grade Level

In [76]:
df_9thgrade=df_students[df_students['grade']=='9th']
df_10thgrade=df_students[df_students['grade']=='10th']
df_11thgrade=df_students[df_students['grade']=='11th']
df_12thgrade=df_students[df_students['grade']=='12th']
grouped_9th=df_9thgrade.groupby('school')
summary_9th=grouped_9th['math_score'].agg(np.mean)
grouped_10th=df_10thgrade.groupby('school')
summary_10th=grouped_10th['math_score'].agg(np.mean)
grouped_11th=df_11thgrade.groupby('school')
summary_11th=grouped_11th['math_score'].agg(np.mean)
grouped_12th=df_12thgrade.groupby('school')
summary_12th=grouped_12th['math_score'].agg(np.mean)
math_summary_bygrade=pd.concat([summary_9th,summary_10th,summary_11th,summary_12th],axis=1)


In [77]:
math_summary_bygrade
# summary_10th

,math_score,math_score,math_score,math_score
school,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


# Average Reading Score at each Grade Level

In [78]:
df_9thgrade=df_students[df_students['grade']=='9th']
df_10thgrade=df_students[df_students['grade']=='10th']
df_11thgrade=df_students[df_students['grade']=='11th']
df_12thgrade=df_students[df_students['grade']=='12th']
grouped_9th=df_9thgrade.groupby('school')
summary_9th=grouped_9th['reading_score'].agg(np.mean)
grouped_10th=df_10thgrade.groupby('school')
summary_10th=grouped_10th['reading_score'].agg(np.mean)
grouped_11th=df_11thgrade.groupby('school')
summary_11th=grouped_11th['reading_score'].agg(np.mean)
grouped_12th=df_12thgrade.groupby('school')
summary_12th=grouped_12th['reading_score'].agg(np.mean)
reading_summary_bygrade=pd.concat([summary_9th,summary_10th,summary_11th,summary_12th],axis=1)

In [79]:
reading_summary_bygrade

,reading_score,reading_score,reading_score,reading_score
school,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


# Scores by School Spending

In [93]:
bins=[570,590,610,630,650,670]
group_names=["570-590","590-610","610-630","630-650","650-670"]
spending_groups=pd.cut(school_summary_df1['Per Student Budget'],bins,labels=group_names)
school_summary_df1['Average Spending Range']=spending_groups
school_summary_df2=school_summary_df1[["Average Spending Range","Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","Overall Passing Rate"]]
grped_df2=school_summary_df2.groupby('Average Spending Range')
school_summary_df3=grped_df2.agg(np.mean)
school_summary_df3


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Average Spending Range,,,,,
570-590,83.455399,83.933814,93.460096,96.610877,95.035486
590-610,83.599686,83.885211,94.230858,95.900287,95.065572
610-630,80.199966,82.425360,80.036217,89.536122,84.786170
630-650,78.229576,81.492857,71.998877,83.757921,77.878399
650-670,76.959583,81.058567,66.218444,81.089710,73.654077


# Scores by School Size

In [95]:
bins=[400,1000,2000,3000,4000,5000]
group_names=["400-1000","1000-2000","2000-3000","3000-4000","4000-5000"]
size_groups=pd.cut(school_summary_df1['Total Students'],bins,labels=group_names)
school_summary_df1['Size Group Range']=size_groups
school_summary_df4=school_summary_df1[["Size Group Range","Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","Overall Passing Rate"]]
grped_df_size=school_summary_df4.groupby('Size Group Range')
school_summary_df5=grped_df_size.agg(np.mean)
school_summary_df5


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Size Group Range,,,,,
400-1000,83.821598,83.929843,93.550225,96.099437,94.824831
1000-2000,83.374684,83.864438,93.599695,96.790680,95.195187
2000-3000,78.429493,81.769122,73.462428,84.473577,78.968003
3000-4000,76.842711,80.744686,66.366592,80.220055,73.293323
4000-5000,77.136883,80.978256,66.496861,81.339570,73.918215


# Scores by School Types

In [97]:
school_summary_type=school_summary_df1[["School Type","Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","Overall Passing Rate"]]


group_type=school_summary_type.groupby("School Type")
school_performance_type=group_type.agg(np.mean)
school_performance_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
